In [ ]:
# 📦 Install necessary libraries
!pip install vaderSentiment nltk scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 5.9 MB/s eta 0:00:00


In [ ]:
# 📚 Import libraries
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
# Download NLTK resources
nltk.download("punkt")
nltk.download('punkt_tab')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#  Load your cleaned reviews file
df = pd.read_csv('clean_bank_reviews.csv')

In [ ]:
#  Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    score = analyzer.polarity_scores(str(text))['compound']
    label = 'positive' if score >= 0.05 else 'negative' if score <= -0.05 else 'neutral'
    return pd.Series([score, label])

df[['sentiment_score', 'sentiment_label']] = df['review'].apply(get_sentiment)

In [ ]:
# Text Preprocessing
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

df['cleaned_review'] = df['review'].apply(preprocess_text)

In [ ]:
# 🔍 TF-IDF Keyword Extraction per Bank
themes = {}

for bank in df['bank'].unique():
    bank_reviews = df[df['bank'] == bank]['cleaned_review']
    tfidf = TfidfVectorizer(ngram_range=(1, 2), max_df=0.85, min_df=2, stop_words='english')
    X = tfidf.fit_transform(bank_reviews)
    scores = zip(tfidf.get_feature_names_out(), X.sum(axis=0).A1)
    top_keywords = sorted(scores, key=lambda x: -x[1])[:10]
    themes[bank] = [kw for kw, _ in top_keywords]

In [ ]:
# Thematic Clustering (Rule-Based)
def assign_theme(text):
    text = text.lower()
    if any(k in text for k in ['login', 'access', 'password', 'otp']):
        return 'Account Access Issues'
    elif any(k in text for k in ['slow', 'transfer', 'crash', 'freeze', 'loading']):
        return 'Transaction Performance'
    elif any(k in text for k in ['interface', 'ui', 'design']):
        return 'User Interface'
    elif any(k in text for k in ['support', 'help', 'response']):
        return 'Customer Support'
    elif any(k in text for k in ['feature', 'update', 'add', 'new']):
        return 'Feature Request'
    else:
        return 'Other'

df['theme'] = df['cleaned_review'].apply(assign_theme)

In [ ]:
# Save Annotated CSV
df.to_csv('annotated_reviews_task2.csv', index=False)
print("✅ Task 2 completed: Saved as 'annotated_reviews_task2.csv'")

✅ Task 2 completed: Saved as 'annotated_reviews_task2.csv'


In [ ]:
#  Show theme summary
df.groupby(['bank', 'theme', 'sentiment_label']).size().unstack(fill_value=0)

sentiment_label                 negative  neutral  positive
bank   theme                                               
BOA    Account Access Issues           7        2         4
       Customer Support                1        0         6
       Feature Request                 5        4         4
       Other                          85      135       107
       Transaction Performance        17       11        14
       User Interface                  1        3         4
CBE    Account Access Issues           1        1         6
       Customer Support                0        0         5
       Feature Request                 2        7         5
       Other                          28       99       211
       Transaction Performance         4        7         8
       User Interface                  0        2         4
Dashen Account Access Issues           2        1        11
       Customer Support                0        0         8
       Feature Request                 0        3        31
       Other                          18       73       183
       Transaction Performance         5        6        13
       User Interface                  1        1        22

In [ ]:
# ✅ KPI CHECK 2 — Themes per bank
print("\n🔍 Number of themes per bank (unique themes used):")
print(df.groupby('bank')['theme'].nunique())


🔍 Number of themes per bank (unique themes used):
bank
BOA       6
CBE       6
Dashen    6
Name: theme, dtype: int64


In [ ]:
#  Example: Show Top 5 keywords per bank
print("\n Top Keywords Extracted Per Bank:")
for bank, keywords in themes.items():
    print(f"\n{bank}:")
    print(", ".join(keywords))


 Top Keywords Extracted Per Bank:

CBE:
app, good, best, nice, cbe, bank, like, good app, great, application

BOA:
app, good, work, bank, boa, working, doesnt, banking, worst, mobile

Dashen:
app, best, dashen, bank, super, good, banking, dashen bank, amazing, use
